Generating figures from GUIDEseq data processed in guideseq_2024_data to characterize Cas9 CRISPRko off-target behavior. 

[Source paper](https://www.biorxiv.org/content/10.1101/2023.11.01.565099v2)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from scipy import stats
import gpplot as gpp 
from statannotations.Annotator import Annotator
from scipy.stats import pearsonr


gpp.set_aesthetics(context = 'paper')


In [2]:
#guideseq collated data with duplicates/on-target sites removed, bulge sites removed, CFD and # mismatches in SDR calculated 
guideseq_nobulges= pd.read_csv("../Data/guideseq_2024_nobulges_withCFD.csv")


In [3]:
guideseq_nobulges

,sgRNA,Align.off-target,Align.#Mismatches,normalized_reads,mismatches_in_SDR,cfd_score
0,GAAAAAGTACACGCCTACAGNGG,GAAAAAGTACACGCCTACAATGG,1.0,0.334686,1,0.937500
1,GAACACAAAGCATAGACTGCNGG,GTACACAAAGCTCAGACTGGCTA,6.0,0.000000,5,0.000000
2,GAACACAAAGCATAGACTGCNGG,GAACTGAAAGCAGAGAATGAAAG,6.0,0.000000,6,0.002869
3,GAACACAAAGCATAGACTGCNGG,GAATAAAAGGCAAAGACAACTGG,6.0,0.000000,6,0.163265
4,GAACACAAAGCATAGACTGCNGG,GAGCAAAGACCAGAGACTGCTGA,6.0,0.000000,5,0.003015
...,...,...,...,...,...,...
3393174,GTTTGCGACTCTGACAGAGCNGG,CTTTGCGGCTCTCACAGTGGTGG,5.0,0.000000,4,0.006487
3393175,GTTTGCGACTCTGACAGAGCNGG,GTTTTGGCCTCTAAAAGTGCCGG,6.0,0.000000,6,0.006593
3393176,GTTTGCGACTCTGACAGAGCNGG,GATTACAACTTTGAAAGAGCCGA,6.0,0.000000,5,0.003482
3393177,GTTTGCGACTCTGACAGAGCNGG,TTTGGAAACTCTGACAGCGCTGC,6.0,0.000000,5,0.002457


**Off-Target Sites with 2 or more mismatches are unlikely to be active**



In [4]:
#get bin of normalized read count 
norm_read_bins = pd.IntervalIndex.from_tuples([(0.5,np.inf),(0.05, 0.5),(0.01, 0.05),(0, 0.01),(-np.inf,0)])
#guideseq_nobulges["normalized_reads_bin"]=pd.cut(guideseq_nobulges["normalized_reads"], bins)
intervals = norm_read_bins.values
interval_labels = ["50+%","5-50%","1-5%","0-1%","0%"]
norm_read_bins_to_name = {interval: label for interval, label in zip(intervals, interval_labels)}
guideseq_nobulges["normalized_reads_bin"] = pd.Series(
    pd.CategoricalIndex(pd.cut(guideseq_nobulges["normalized_reads"], norm_read_bins)).rename_categories(norm_read_bins_to_name))


In [5]:
guideseq_0mm= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]==0)].reset_index(drop=True).copy()
guideseq_1mm= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]==1)].reset_index(drop=True).copy()
guideseq_2mm= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]==2)].reset_index(drop=True).copy()
guideseq_3mm= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]==3)].reset_index(drop=True).copy()
guideseq_4plusmm= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]>=4)].reset_index(drop=True).copy()



In [6]:
#fraction of OTS in each category that are "active": has at least 1% of reads as on-target site 
guideseq_0mm_fraction_active= len(guideseq_0mm[guideseq_0mm["normalized_reads"]>0.01])/len(guideseq_0mm)
guideseq_1mm_fraction_active= len(guideseq_1mm[guideseq_1mm["normalized_reads"]>0.01])/len(guideseq_1mm)
guideseq_2mm_fraction_active= len(guideseq_2mm[guideseq_2mm["normalized_reads"]>0.01])/len(guideseq_2mm)
guideseq_3mm_fraction_active= len(guideseq_3mm[guideseq_3mm["normalized_reads"]>0.01])/len(guideseq_3mm)
guideseq_4plusmm_fraction_active= len(guideseq_4plusmm[guideseq_4plusmm["normalized_reads"]>0.1])/len(guideseq_4plusmm)

In [7]:
#average normalized reads at active OTS 
active_0mm_avg_norm_reads= guideseq_0mm[guideseq_0mm["normalized_reads"]>0.01]["normalized_reads"].mean()
active_1mm_avg_norm_reads= guideseq_1mm[guideseq_1mm["normalized_reads"]>0.01]["normalized_reads"].mean()
active_2mm_avg_norm_reads= guideseq_2mm[guideseq_2mm["normalized_reads"]>0.01]["normalized_reads"].mean()
active_3mm_avg_norm_reads= guideseq_3mm[guideseq_3mm["normalized_reads"]>0.01]["normalized_reads"].mean()
active_4plusmm_avg_norm_reads= guideseq_4plusmm[guideseq_4plusmm["normalized_reads"]>0.01]["normalized_reads"].mean()

In [8]:
#get info for bulges
guideseq_bulges= pd.read_csv("../Data/guideseq_bulges.csv")
guideseq_bulge_fraction_active= len(guideseq_bulges[guideseq_bulges["normalized_reads"]>0.01])/len(guideseq_bulges)
active_bulge_avg_norm_reads= guideseq_bulges[guideseq_bulges["normalized_reads"]>0.01]["normalized_reads"].mean()
guideseq_bulges["normalized_reads_bin"] = pd.Series(
    pd.CategoricalIndex(pd.cut(guideseq_bulges["normalized_reads"], norm_read_bins)).rename_categories(norm_read_bins_to_name))
guideseq_bulges

OSError: [Errno 89] Operation canceled

In [ ]:

#gets counts of OTS in each normalized read count bin, stratified by MM# 

guideseq_0mm_bincounts= pd.DataFrame(guideseq_0mm["normalized_reads_bin"].value_counts()).reset_index()
guideseq_0mm_bincounts["normalized_reads_bin_frac"]=guideseq_0mm_bincounts["count"]/len(guideseq_0mm)

guideseq_1mm_bincounts= pd.DataFrame(guideseq_1mm["normalized_reads_bin"].value_counts()).reset_index()
guideseq_1mm_bincounts["normalized_reads_bin_frac"]=guideseq_1mm_bincounts["count"]/len(guideseq_1mm)

guideseq_2mm_bincounts= pd.DataFrame(guideseq_2mm["normalized_reads_bin"].value_counts()).reset_index()
guideseq_2mm_bincounts["normalized_reads_bin_frac"]=guideseq_2mm_bincounts["count"]/len(guideseq_2mm)

guideseq_3mm_bincounts= pd.DataFrame(guideseq_3mm["normalized_reads_bin"].value_counts()).reset_index()
guideseq_3mm_bincounts["normalized_reads_bin_frac"]=guideseq_3mm_bincounts["count"]/len(guideseq_3mm)

guideseq_4plusmm_bincounts= pd.DataFrame(guideseq_4plusmm["normalized_reads_bin"].value_counts()).reset_index()
guideseq_4plusmm_bincounts["normalized_reads_bin_frac"]=guideseq_4plusmm_bincounts["count"]/len(guideseq_4plusmm)

guideseq_bulges_bincounts= pd.DataFrame(guideseq_bulges["normalized_reads_bin"].value_counts()).reset_index()
guideseq_bulges_bincounts["normalized_reads_bin_frac"]=guideseq_bulges_bincounts["count"]/len(guideseq_bulges)

norm_read_bincounts_all=pd.concat([guideseq_0mm_bincounts,guideseq_1mm_bincounts,guideseq_2mm_bincounts,guideseq_3mm_bincounts,guideseq_4plusmm_bincounts,guideseq_bulges_bincounts],keys=["0MM","1MM","2MM","3MM","4+MM","Bulge"]).reset_index()
norm_read_bincounts_all_pivot = norm_read_bincounts_all.pivot(index='level_0', columns='normalized_reads_bin', values='normalized_reads_bin_frac')


In [ ]:
(fig,ax)=plt.subplots(1,2,figsize=(14,6))
fig.suptitle("GUIDE-seq of off-target sites (OTS) for 114 sgRNAs (Yaish & Orenstein, 2024)",size=16)
fig.supxlabel("# Mismatches in SDR to target site",size=16)

num_ots_mm= pd.DataFrame(data={"Off-Target Site Type":["0","1","2","3","4+","N/A (Bulge)"],"# Sites":[len(guideseq_0mm),len(guideseq_1mm),len(guideseq_2mm),len(guideseq_3mm),len(guideseq_4plusmm),len(guideseq_bulges)]})
sns.barplot(data=num_ots_mm,x="Off-Target Site Type",y="# Sites",ax=ax[0],color="lightgrey")
ax[0].bar_label(ax[0].containers[0], fmt = '%d',size=14)
ax[0].set_yticks([])
ax[0].set_xticks(ticks=[0,1,2,3,4,5],labels=["0","1","2","3","4+","Bulge"],fontsize=16)
ax[0].set_xlabel("",size=16)
ax[0].set_ylabel("",size=16)
#line below makes sure graph is tall enough so that the text doesn't get cut off
ax[0].set_ylim(0,num_ots_mm["# Sites"].max()*1.1)
ax[0].set_xlim(-0.5,5.7)
ax[0].set_title("# potential OTS",size=16)

norm_read_bincounts_all_pivot.plot(kind='bar', stacked=True,ax=ax[1],edgecolor="none",cmap="GnBu_r")
ax[1].set_xticks(ticks=[0,1,2,3,4,5],labels=["0","1","2","3","4+","Bulge"],fontsize=16,rotation=0)
ax[1].set_xlabel("",size=16)
ax[1].set_xlabel("",size=16)

handles,labels = ax[1].get_legend_handles_labels()
sorted_legends= [x for _,x in sorted(zip(interval_labels,labels))] 
sorted_handles=[x for _,x in sorted(zip(interval_labels,handles))]
ax[1].legend(sorted_handles,sorted_legends, loc='upper right',fontsize=16)
ax[1].set_ylabel("Fraction of potential OTS",size=16)
ax[1].set_ylim(0,1.05)
ax[1].set_title("Reads at potential OTS (% relative to target site) ",size=16)

gpp.savefig("../Figures/ots_count_and_guideseq_activity_breakdown.pdf", dpi=600, bbox_inches = 'tight')


In [ ]:
#identify fraction of OTS at each mismatch count that are not guideseq active (at least 0.1 normalized reads)
get_fnr=norm_read_bincounts_all_pivot.copy()
get_fnr["fraction active"]=get_fnr[["1-5%","5-50%","50+%"]].sum(axis=1)
get_fnr

**CFD effectively predicts the probability of activity at off-target sites**


In [ ]:
#indicates whether or not an off-target site is at least 1% as active as the on-target site 
guideseq_nobulges["notably_active"]=guideseq_nobulges["normalized_reads"]>0.01
guideseq_nobulges["cfd_x100"]=(guideseq_nobulges["cfd_score"]*100).astype(int)


In [ ]:
num_bins=150
below_3MM= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]<3)].reset_index(drop=True).copy()
num_OTS_per_bin_below3MM=len(below_3MM)/num_bins
num_sgRNAs=len(below_3MM["sgRNA"].unique())
label=str(int(num_OTS_per_bin_below3MM))+" OTS"
below_3MM["CFD_interval"]=pd.qcut(below_3MM["cfd_x100"],num_bins,duplicates='drop')
below_3MM["CFD_interval_midpoint"]=below_3MM["CFD_interval"].apply(lambda x: x.mid*0.01) 

#for error bars
below_3MM["CFD_low_error"]=below_3MM["CFD_interval"].apply(lambda x: x.mid*0.01-x.left*0.01) 
below_3MM["CFD_high_error"]=below_3MM["CFD_interval"].apply(lambda x: x.right*0.01-x.mid*0.01) 
cfd_bin_extremes=below_3MM[["CFD_low_error","CFD_high_error","CFD_interval_midpoint"]].drop_duplicates(keep="first").sort_values(by="CFD_interval_midpoint")
bin_cfd_low_errors=cfd_bin_extremes["CFD_low_error"].tolist()
bin_cfd_high_errors=cfd_bin_extremes["CFD_high_error"].tolist()

binned_cfd_and_effective= below_3MM[["CFD_interval_midpoint","notably_active"]].groupby("CFD_interval_midpoint",sort=True,as_index=False,observed=True)
fraction_effective_binned_below3MM= binned_cfd_and_effective.mean()



In [ ]:
label=str(int(num_OTS_per_bin_below3MM))+" OTS"
plt.scatter(data=fraction_effective_binned_below3MM,y="CFD_interval_midpoint",x="notably_active",c="cornflowerblue",s=60,label=label)
plt.legend(fontsize=16,loc="upper left")
plt.title("Predicted vs. Experimental OTS activity",fontsize=16)
plt.ylabel("Median CFD of OTS in bin",fontsize=16)
plt.xlabel("Fraction GUIDE-seq active",fontsize=16)
plt.gca().set_aspect('equal')
#adding errorbars with 5th, 95th CFD percentile in bin
plt.errorbar(fraction_effective_binned_below3MM["CFD_interval_midpoint"], fraction_effective_binned_below3MM["notably_active"], yerr=None,xerr = [bin_cfd_low_errors,bin_cfd_high_errors],ls="none",c="grey") 
plt.xlim(0,1)
plt.ylim(0,1)
r,_=pearsonr(fraction_effective_binned_below3MM["notably_active"], fraction_effective_binned_below3MM["CFD_interval_midpoint"])
plt.text(0.6, 0.1, f"r = {r:.2f}" , fontsize=16, color='black')
plt.tick_params(axis='both', which='major', labelsize=14)
gpp.savefig("../Figures/correlation_cfd_guideseq_activity.pdf", dpi=600, bbox_inches = 'tight')


## Contrast CFD of active vs inactive OTS

In [ ]:
guideseq_nobulges
binorder= [False,True]
box_pairs=[(False,True)]

ax = sns.boxplot(x=guideseq_nobulges['notably_active'], y=guideseq_nobulges['cfd_score'],color = 'cornflowerblue')
plt.ylabel("CFD Score",fontsize=16)
plt.xlabel("Active (>=0.01 normalized reads) in GUIDE-seq data", fontsize=16)
plt.title("CFD Score of active vs. inactive OTS in GUIDE-seq data",fontsize=16)

annotator = Annotator(ax, [(False,True)], data=guideseq_nobulges, x="notably_active", y='cfd_score', order=[False,True],verbose=False)
annotator.configure(test='Mann-Whitney-ls', text_format='simple',fontsize=16) #this shows test name, actual p value
annotator.apply_and_annotate()

gpp.savefig("../Figures/cfd_active_v_inactive.pdf", dpi=600, bbox_inches = 'tight')


In [ ]:
guideseq_nobulges["mismatches_in_SDR"].value_counts()

### OTS Considered in Aggregate CFD vs. Guidescan

Aggregate CFD: <2 MM OTS (MM counted in the SDR, which includes the PAM)

Guidescan: <4 MM OTS (MM counted in 20nt guide sequence)

In [ ]:
potential_aggcfd_ots= guideseq_nobulges[(guideseq_nobulges["mismatches_in_SDR"]<2)].reset_index()
potential_guidescan_ots= guideseq_nobulges[(guideseq_nobulges["Align.#Mismatches"]<4)].reset_index()

print("average # OTS in specificity score per guide:",len(potential_guidescan_ots)/len(guideseq_nobulges["sgRNA"].unique()))
print("average # OTS in aggregate CFD per guide:",len(potential_aggcfd_ots)/len(guideseq_nobulges["sgRNA"].unique()))


In [ ]:
potential_aggcfd_ots_bincounts= pd.DataFrame(potential_aggcfd_ots["normalized_reads_bin"].value_counts()).reset_index()
potential_aggcfd_ots_bincounts["normalized_reads_bin_frac"]=potential_aggcfd_ots_bincounts["count"]/len(potential_aggcfd_ots)

potential_guidescan_ots_bincounts= pd.DataFrame(potential_guidescan_ots["normalized_reads_bin"].value_counts()).reset_index()
potential_guidescan_ots_bincounts["normalized_reads_bin_frac"]=potential_guidescan_ots_bincounts["count"]/len(potential_guidescan_ots)


In [ ]:
norm_read_bincounts_both=pd.concat([potential_aggcfd_ots_bincounts,potential_guidescan_ots_bincounts],keys=["<2MM in SDR","<4MM in 20nt sequence\nNGG PAM only"]).reset_index()
norm_read_bincounts_both_pivot = norm_read_bincounts_both.pivot(index='level_0', columns='normalized_reads_bin', values='normalized_reads_bin_frac')
ax=norm_read_bincounts_both_pivot.plot(kind='bar', stacked=True,edgecolor="none",cmap="GnBu_r")
plt.xticks(ticks=[0,1],labels=["<2MM in SDR","<4MM in 20nt sequence"],fontsize=16,rotation=0)
plt.xlabel("OTS Set",fontsize=16)
plt.ylabel("Fraction of OTS in set",fontsize=16)
handles,labels = ax.get_legend_handles_labels()
sorted_legends= [x for _,x in sorted(zip(interval_labels,labels))] 
sorted_handles=[x for _,x in sorted(zip(interval_labels,handles))]
plt.legend(sorted_handles,sorted_legends, title="# Reads at off-target site\n relative to target site",bbox_to_anchor=(1,1),fontsize=14,title_fontsize=14)
plt.title("Activity at OTS for 114 GUIDE-seq sgRNAs",size=16)
gpp.savefig("../Figures/GUIDEseq_activity_by_OTS_subset.pdf", dpi=600, bbox_inches = 'tight')

In [ ]:
print("Agg CFD OTS FDR:",len(potential_aggcfd_ots[(potential_aggcfd_ots["normalized_reads_bin"]=="0%")|(potential_aggcfd_ots["normalized_reads_bin"]=="0-1%")])/len(potential_aggcfd_ots))
print("Guidescan OTS FDR:",len(potential_guidescan_ots[(potential_guidescan_ots["normalized_reads_bin"]=="0%")|(potential_guidescan_ots["normalized_reads_bin"]=="0-1%")])/len(potential_guidescan_ots))

In [ ]:
total_active_ots=len(guideseq_nobulges[~guideseq_nobulges["normalized_reads_bin"].isin(["0%","0-1%"])])
print("% of active OTS captured by agg CFD OTS:",100*len(potential_aggcfd_ots[~potential_aggcfd_ots["normalized_reads_bin"].isin(["0%","0-1%"])])/total_active_ots)
print("% of active OTS captured by Guidescan OTS:",100*len(potential_guidescan_ots[~potential_guidescan_ots["normalized_reads_bin"].isin(["0%","0-1%"])])/total_active_ots)